05 出力層の設計
=============

* ニューラルネットワークは、`分類問題`と`回帰問題`の両方に用いることができる

    * ただし、それぞれの問題に対して出力層の活性化関数を変更する必要がある
    
    * 一般的に、`回帰問題`では`恒等関数`を、`分類問題`では`ソフトマックス関数`を用いる

## 1. 恒等関数とソフトマックス関数

* `恒等関数`：入力をそのまま出力する

    * 入ってきたものに対して何も手を加えずに出力する
    
    * そのため、出力層で恒等関数を用いるときは、入力信号をそのまま出力するだけになる
    
    * 恒等関数によって変換されるプロセスは、これまでの隠れ層での活性化関数と同じで、1本の矢印で描画する
    
![恒等関数](./images/恒等関数.png)

* `ソフトマックス関数`：出力層が全部で$n$個あるとして、$k$番目の出力$y_k$を求める

    * 分子は入力信号$a_k$の指数関数、分母は全ての入力信号の指数関数の和から構成される
    
\begin{eqnarray}
y_k = \frac{\exp (a_k)}{\sum_{i=1}^{n} \exp (a_i)}
\end{eqnarray}

* `ソフトマックス関数`を図で表すと、次の図のようになる

    * 全ての入力信号から矢印による結びつきがある
    
    * 分母からわかるように、出力の各ニューロンが、全ての信号から影響を受けることになる
    
![ソフトマックス関数](./images/ソフトマックス関数.png)

* 実際に、`ソフトマックス関数`を実装してみる

In [1]:
import numpy as np

a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)
print(exp_a)

[ 1.34985881 18.17414537 54.59815003]


In [2]:
sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

74.1221542101633


In [3]:
y = exp_a / sum_exp_a
print(y)

[0.01821127 0.24519181 0.73659691]


* この実装は、関数として次のように定義する

In [4]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

## 2. ソフトマックスの実装上の注意

* `softmax`関数は式を正しく表現できているが、コンピュータでの計算を行う際には欠陥がある

    * 原因として、`オーバーフロー`が挙げられる
    
    * 指数関数において、$e^{1000}$などの値になった場合、無限大を示す`inf`が返ってくる
    
    * また、大きな値同士で割り算を行うと、数値が"不安定"な結果になってしまう

* `ソフトマックス関数`の実装の改善案は、以下の式から導出される

\begin{eqnarray}
y_k = \frac{\exp (a_k)}{\sum_{i=1}^{n} \exp (a_i)} = \frac{C\exp (a_k)}{C\sum_{i=1}^{n} \exp (a_i)} \\
= \frac{\exp (a_k + \log C)}{\sum_{i=1}^{n} \exp (a_i + \log C)} \\
= \frac{\exp (a_k + C')}{\sum_{i=1}^{n} \exp (a_i + C')}
\end{eqnarray}

* ソフトマックスの指数関数の計算を行う際には、何らかの定数を足し算(もしくは引き算)しても結果が変わらない

    * ここの$C'$にはどのような値も入れることができるが、オーバーフローの対策には入力信号の中でも最大の値を用いる

In [5]:
a = np.array([1010, 1000, 990])

# 正しく計算されない
np.exp(a) / np.sum(np.exp(a))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


array([nan, nan, nan])

In [6]:
c = np.max(a)
print(c)

1010


In [7]:
a - c

array([  0, -10, -20])

In [8]:
np.exp(a - c) / np.sum(np.exp(a - c))

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

* この例より、普通に計算していたら`nan`が出力されるが、入力信号の最大値(`c`)を引くことで、正しく計算されることがわかる

    * 以上のことを含めて`ソフトマックス関数`を実装すると、次のようになる

In [9]:
def softmax(a):
    c = np.max(a)
    # オーバーフロー対策
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

## 3. ソフトマックス関数の特徴

* `softmax()`関数を使うことで、ニューラルネットワークの出力は次のように計算することができる

In [10]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)
print(y)

[0.01821127 0.24519181 0.73659691]


In [11]:
np.sum(y)

1.0

* ここで示したように、`ソフトマックス関数`の出力は`0`から`1.0`の間の実数になる

    * また、`ソフトマックス関数`の出力の総和は`1`となる
    
    * この性質により、`ソフトマックス関数`の出力を「確率」として解釈することができる
    
    * 例)`y[0]`の確率は0.018、`y[1]`の確率は0.245、`y[2]`の確率は0.737
    
    * この結果により、「2番目の要素が最も確率が高いため、答えは2番目のクラス」と言うことができる
    
    * また、「74%の確率で2番目のクラス、25%の確率で1番目のクラス、1%の確率で0番目のクラス」と言う表現もできる
    

* ここで注意点として、`ソフトマックス関数`を適用しても各要素の大小関係は変わらないこと

    * これは、指数関数が単調増加する関数であることに起因する
    
    * 実際、上の例では`a`の要素の大小関係と`y`の要素の大小関係は変わっていない

* ニューラルネットワークのクラス分類では、一般的に、出力の一番大きいニューロンに相当するクラスだけを認識結果とする

    * そして、`ソフトマックス関数`を適用しても、出力の一番大きいニューロンの位置は変わらない
    
    * そのため、ニューラルネットワークが分類を行う際には、出力層のソフトマックス関数を省略することができる
    
* 実際の問題では、指数関数の計算は、それなりにコンピュータの計算が必要になるので、出力層のソフトマックス関数は省略するのが一般的

## 4. 出力層のニューロンの数

* 出力層のニューロンの数は、解くべき問題に応じて、適宜決める必要がある

* クラス分類を行う問題では、出力層のニューロンの数は分類したいクラスの数に設定するのが一般的

    * 例)ある入力画像に対して、その画像が数字の`0`から`9`のどれかを予測(10クラス分類問題)では、以下の図のように出力層のニューロンは10個に設定

![出力層のニューロンは各数字に対応する](./images/出力層のニューロンは各数字に対応する.png)

* この例では、出力層のニューロンは上から順に数字の`0`、`1`、...、`9`に対応する

    * また、この図では$y_2$が一番濃く描画されており、$y_2$のニューロンが一番高い値を出力している
    
    * これは、$y_2$に該当するクラス、つまり「2」であることを、このニューラルネットワークが予測している

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/05/01 |